In [25]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intent.json').read()
intents = json.loads(data_file)

In [26]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [27]:
words

['Hi',
 'there',
 'How',
 'are',
 'you',
 'Is',
 'anyone',
 'there',
 '?',
 'Hey',
 'Hola',
 'Hello',
 'Good',
 'day',
 'Sup',
 'What',
 "'s",
 'up',
 '?',
 'I',
 "'m",
 'bored',
 'Bye',
 'See',
 'you',
 'later',
 'Goodbye',
 'Nice',
 'chatting',
 'to',
 'you',
 ',',
 'bye',
 'Till',
 'next',
 'time',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Awesome',
 ',',
 'thanks',
 'Thanks',
 'for',
 'helping',
 'me',
 'How',
 'you',
 'could',
 'help',
 'me',
 '?',
 'What',
 'you',
 'can',
 'do',
 '?',
 'What',
 'help',
 'you',
 'provide',
 '?',
 'How',
 'you',
 'can',
 'be',
 'helpful',
 '?',
 'What',
 'support',
 'is',
 'offered',
 'Can',
 'you',
 'suggest',
 'me',
 'a',
 'thriller',
 'book',
 '?',
 'Thriller',
 'Crime',
 'Mystery',
 'Can',
 'you',
 'suggest',
 'more',
 '?',
 'No',
 'not',
 'this',
 'Can',
 'you',
 'suggest',
 'a',
 'horror',
 'book',
 'scary',
 'horror',
 'book',
 'No',
 'some',
 'other',
 'book',
 'please',
 'I',
 'want',
 'to',
 'read',
 'a',
 'romance',
 'bo

In [28]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

51 documents
10 classes ['History ', 'Horror', 'Romance', 'Thriller', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
82 unique lemmatized words ["'m", "'s", ',', 'a', 'anyone', 'are', 'awesome', 'be', 'book', 'bored', 'bye', 'can', 'chatting', 'could', 'crime', 'data', 'day', 'detail', 'do', 'find', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'horror', 'hospital', 'how', 'i', 'is', 'later', 'list', 'locate', 'looking', 'lookup', 'love', 'me', 'more', 'mystery', 'nearby', 'next', 'nice', 'no', 'not', 'of', 'offered', 'other', 'patient', 'pharmacy', 'please', 'provide', 'read', 'romance', 'scary', 'search', 'searching', 'see', 'some', 'suggest', 'sup', 'support', 'sweet', 'thank', 'thanks', 'that', 'there', 'this', 'thriller', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'you']


In [29]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-29-451aefdaa425>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [30]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
11/11 [==============================] - 0s 5ms/step - loss: 2.3203 - accuracy: 0.0588
Epoch 2/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2073 - accuracy: 0.2353
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2631 - accuracy: 0.2157
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 2.1487 - accuracy: 0.2549
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 2.0510 - accuracy: 0.3529
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 1.9646 - accuracy: 0.3725
Epoch 7/200
11/11 [==============================] - 0s 3ms/step - loss: 1.9670 - accuracy: 0.3922
Epoch 8/200
11/11 [==============================] - 0s 3ms/step - loss: 1.8414 - accuracy: 0.3333
Epoch 9/200
11/11 [==============================] - 0s 2ms/step - loss: 1.7849 - accuracy: 0.4510
Epoch 10/200
11/11 [==============================] - 0s 2ms/step - loss: 1.6489 - accuracy: 0.5098
Epoch 11/

11/11 [==============================] - 0s 2ms/step - loss: 0.0461 - accuracy: 1.0000
Epoch 84/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1144 - accuracy: 0.9804
Epoch 85/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0344 - accuracy: 1.0000
Epoch 86/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0446 - accuracy: 1.0000
Epoch 87/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0660 - accuracy: 0.9804
Epoch 88/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1086 - accuracy: 0.9804
Epoch 89/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0500 - accuracy: 0.9804
Epoch 90/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0310 - accuracy: 1.0000
Epoch 91/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0648 - accuracy: 1.0000
Epoch 92/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0547 - accuracy: 0.9804
Epoch 93/200
